# A Closed Form Reduced Order Electrochemical Model for Lithium-Ion Cells

In [1]:
import CaseStudy.AshwiniModel as am
using Gadfly

In [2]:
I = 13.5
t = Float64.(0:1:25000)
tspan = 10

10

In [3]:
# Start model solving

# Calculating initial value of k₂


# dₚ₁ᵢ = am.dₚ₁₀(I)